# trade의 데이터 형태 분석
{<br>
&nbsp;&nbsp;&nbsp;&nbsp;"type":"trade",&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;      토픽을 결정짓는 attr <br>
&nbsp;&nbsp;&nbsp;&nbsp;"code":"KRW-BTC",&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    실제로 카운팅을 진행하여 인사이트를 뽑아내는 필드<br>
&nbsp;&nbsp;&nbsp;&nbsp;"timestamp":1575553985422,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  전송 타임스탬프 <br>
&nbsp;&nbsp;&nbsp;&nbsp;"trade_date":"2019-12-05",&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  거래 날짜 - 거래 타임스탬프가 있어서 필요없음 <br>
&nbsp;&nbsp;&nbsp;&nbsp;"trade_time":"13:53:05",&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    거래 시간 - 거래 타임스탬프가 있어서 필요없음 <br>
&nbsp;&nbsp;&nbsp;&nbsp;"trade_timestamp":1575553985000,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   거래 타임스탬프 - 아주 낮은 확률이지만 동시에 같은 시간에 거래가 일어날 수 있어서 식별자의 가치가 떨어짐<br>
&nbsp;&nbsp;&nbsp;&nbsp;"trade_price":8703000.0,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  거래 가격 <br>
&nbsp;&nbsp;&nbsp;&nbsp;"trade_volume":0.0069,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   거래량<br>
&nbsp;&nbsp;&nbsp;&nbsp;"ask_bid":"BID",&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  매수/매도 채결  <br>
&nbsp;&nbsp;&nbsp;&nbsp;"prev_closing_price":8515000.00000000,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  최근 가격<br>
&nbsp;&nbsp;&nbsp;&nbsp;"change":"RISE",&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                        변화<br>
&nbsp;&nbsp;&nbsp;&nbsp;"change_price":188000.00000000,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         변화액<br>
&nbsp;&nbsp;&nbsp;&nbsp;"sequential_id":1575553985000000,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;       거래ID - 식별자로써의 가치(타임스탬프 + 시퀀셜 넘버)<br>
&nbsp;&nbsp;&nbsp;&nbsp;"stream_type":"SNAPSHOT"&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                어차피 스냅샷이니 필요없음<br>
}

In [1]:
!pip3 install confluent_kafka
!pip3 install websocket-client

    100% |################################| 7.4MB 220kB/s eta 0:00:01
    100% |################################| 204kB 1.2MB/s ta 0:00:01


In [2]:
from confluent_kafka import Producer
import sys

In [3]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

In [4]:
def pub_msg(producer,topic,msg):
    try:
        # Produce line (without newline)
        producer.produce(topic, msg, callback=delivery_report)
        producer.poll(0)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' %
                             len(producer))
    except Exception as e:
        print(e)

In [5]:
import websocket
import json

try:
    import thread
except ImportError:
    import _thread as thread
import time

global count 
global producer 
producer = Producer({'bootstrap.servers': 'localhost:9092,localhost:9093,localhost:9094'})

count = 0

def on_message(ws, message):
    global count 
    global producer 
    
    count += 1
    try:
        if count == 100000:
            ws.close()
        pub_msg(producer,'trade',message)
    except Exception:
        ws.close()

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    def run(*args):
        ws.send('[{"ticket":"koock2"},{"type":"trade","codes":["KRW-BTC","KRW-ETH","KRW-BCH","KRW-XRP","ETH-BCH","BTC-BCH"]}]')
    thread.start_new_thread(run, ())


if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("wss://api.upbit.com/websocket/v1",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

--- request header ---
GET /websocket/v1 HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: api.upbit.com
Origin: http://api.upbit.com
Sec-WebSocket-Key: INStPmQkNi7PvAIAeMpGOQ==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Thu, 05 Dec 2019 16:32:37 GMT
Content-Type: text/plain; charset=UTF-8
Connection: upgrade
Limit-By-IP: Yes
Remaining-Req: group=websocket; min=99; sec=4
upgrade: websocket
sec-websocket-accept: FAmUNWNlnlF43EXikLNdV5m1bd8=
-----------------------
send: b'\x81\xecf\x9d\xa9\x08=\xe6\x8b|\x0f\xfe\xc2m\x12\xbf\x93*\r\xf2\xc6k\r\xaf\x8buJ\xe6\x8b|\x1f\xed\xcc*\\\xbf\xddz\x07\xf9\xcc*J\xbf\xcag\x02\xf8\xda*\\\xc6\x8bC4\xca\x84J2\xde\x8b$D\xd6\xfb_K\xd8\xfd@D\xb1\x8bC4\xca\x84J%\xd5\x8b$D\xd6\xfb_K\xc5\xfbXD\xb1\x8bM2\xd5\x84J%\xd5\x8b$D\xdf\xfdKK\xdf\xea@D\xc0\xd4U'


Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message 

Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message 

Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message 

Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message 

Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message delivered to trade [0]
Message 